In [ ]:
# The following code requires up to 30 GB of RAM over 2 hours if using 4 CPUs

In [ ]:
# Import libraries
import importlib
from glob import glob

In [ ]:
# Choose calc_funcs_ver to use in plotting scripts.
cfv = "latest"

assert (isinstance(cfv, str) & (len(cfv) == 5) & (cfv[:3] == "cfv") & 
        cfv[3].isnumeric() & cfv[4].isalpha() & cfv[4].islower()) | (cfv == "latest"), \
    ("cfv must be 'latest' or of form 'cfvXY' where X is a single digit number " +
     "and Y is a lowercase alphabet character. eg. cfv1n")

if cfv == "latest":
    calc_funcs_scripts = glob("calc_funcs_*.py")
    calc_funcs_scripts.sort()
    calc_funcs_module = calc_funcs_scripts[-1][:-3]
    
else:
    calc_funcs_module = "calc_funcs_" + cfv[2:]
    
cf = importlib.import_module(calc_funcs_module)

print(f"Using: {calc_funcs_module}")

In [ ]:
# Choose plot_funcs_ver to use.
pfv = "latest"

assert (isinstance(pfv, str) & (len(pfv) == 5) & (pfv[:3] == "pfv") & 
        pfv[3].isnumeric() & pfv[4].isalpha() & pfv[4].islower()) | (pfv == "latest"), \
    ("pfv must be 'latest' or of form 'pfvXY' where X is a single digit number " +
     "and Y is a lowercase alphabet character. eg. pfv1e")

if pfv == "latest":
    plot_funcs_scripts = glob("plot_funcs_*.py")
    plot_funcs_scripts.sort()
    plot_funcs_module = plot_funcs_scripts[-1][:-3]
    
else:
    plot_funcs_module = "plot_funcs_" + pfv[2:]
    
pf = importlib.import_module(plot_funcs_module)

print(f"Using: {plot_funcs_module}")

In [ ]:
# Create glass rolling avg files to identify years with significant LAI change
cf.calc_glass_rolling_avg_of_annual_diff("wa", 1983, 2019, "all", 5, "avhrr")
cf.calc_glass_rolling_avg_of_annual_diff("wa", 1983, 2019, "all", 5, "modis")

In [ ]:
# Create all possible diff data files for selected periods
# Do this for year round, wet season only, and dry season only
cf.create_all_possible_diff_data_files("wa", "Jan-1994", "Dec-1998", "Jan-2004", "Dec-2008", "all")
cf.create_all_possible_diff_data_files("wa", "Jan-1994", "Dec-1998", "Jan-2004", "Dec-2008", "mam")
cf.create_all_possible_diff_data_files("wa", "Jan-1994", "Dec-1998", "Jan-2004", "Dec-2008", "djf")

In [ ]:
# Create all possible diff data files for selected periods for rabbit-proof fence
# Do this for year round, wet season only, and dry season only
cf.create_all_possible_diff_data_files("wa", "Jan-2019", "Dec-2019", "Jan-2021", "Dec-2021", "all")
cf.create_all_possible_diff_data_files("wa", "Jan-2019", "Dec-2019", "Jan-2021", "Dec-2021", "mam")
cf.create_all_possible_diff_data_files("wa", "Jan-2019", "Dec-2019", "Jan-2021", "Dec-2021", "djf")

In [ ]:
# Plot the glass rolling avg files to identify years with significant LAI change using AVHRR data
pf.create_glass_rolling_plot("wa", 1983, 2019, "all", 5, "mlai", glass_source_pref="avhrr", output=True)

In [ ]:
# Plot the glass rolling avg files to identify years with significant LAI change using MODIS data
pf.create_glass_rolling_plot("wa", 1983, 2019, "all", 5, "mlai", glass_source_pref="modis", output=True)

In [ ]:
# Plot the land surface elevation
pf.create_orog_static_plot("lse", "wa", output=True)

In [ ]:
# Plot the slope of sub-gridscale orography
pf.create_orog_static_plot("ssgo", "wa", output=True)

# Comparison 1

- period1 = Jan-1994 to Dec-1998
- period2 = Jan-2004 to Dec-2008
- extents = [113, 123, -35, -30]
- tz = UTC +8
- glass_data_source = AVHRR

## Comparison 1, year round (all months)

In [ ]:
%%time
pf.create_all_possible_comp_plot_files(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", perc=False, mask_perc_quantile=pf.mask_perc_quantile_default, 
    mask_period1=None, mask_period2=None, extents=None, cfv_data=None
)

### Comparison 1, year round (all months), MDP climatology stats

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="mslp", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="nac", perc=False, 
    mask_perc_quantile=20, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="u100", perc=False, 
    mask_perc_quantile=20, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="v100", perc=False, 
    mask_perc_quantile=20, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="t2", perc=False, 
    mask_perc_quantile=20, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="ws100", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="wv100", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="slhf", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="sshf", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="nse", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="vidmf", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="viec", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="vipile", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="vike", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="tcclw", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="tcwv", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="blh", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="fa", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="cbh", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="tcc", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="cape", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

In [ ]:
pf.plot_comp_mdp_clim_stats_given_var_or_dvar(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all", 
    glass_source_pref="avhrr", var_or_dvar="ci", perc=False, 
    mask_perc_quantile=10, mask_period1=None, 
    mask_period2=None, extents=None, cfv_data=None, output=True
)

### Comparison 1, year round (all months), MDP climatology values

### Comparison 1, year round (all months), WSD climatology

In [ ]:
pf.plot_comp_wsd_clim(
    region="wa", period1_start="Jan-1994", period1_end="Dec-1998", 
    period2_start="Jan-2004", period2_end="Dec-2008", months_subset="all",
    glass_source_pref="avhrr", perc=False, mask_perc_quantile=10, 
    mask_period1=None, mask_period2=None, extents=None, cfv_data=None, output=True
)

## Comparison 1, wet season (MAM months)

### Comparison 1, wet season (MAM months), MDP climatology stats

### Comparison 1, wet season (MAM months), MDP climatology values

### Comparison 1, wet season (MAM months), WSD climatology

## Comparison 1, dry season (DJF months)

### Comparison 1, dry season (DJF months), MDP climatology stats

### Comparison 1, dry season (DJF months), MDP climatology values

### Comparison 1, dry season (DJF months), WSD climatology

# Comparison 2

- period1 = Jan-2019 to Dec-2019
- period2 = Jan-2021 to Dec-2021
- extents = [113, 123, -35, -30] or [117, 122, -35, -30]
- tz = UTC +8
- glass_data_source = MODIS

## Comparison 2, year round (all months)

### Comparison 2, year round (all months), MDP climatology stats

### Comparison 2, year round (all months), MDP climatology values

### Comparison 2, year round (all months), WSD climatology

## Comparison 2, wet season (MAM months)

### Comparison 2, wet season (MAM months), MDP climatology stats

### Comparison 2, wet season (MAM months), MDP climatology values

### Comparison 2, wet season (MAM months), WSD climatology

## Comparison 2, dry season (DJF months)

### Comparison 2, dry season (DJF months), MDP climatology stats

### Comparison 2, dry season (DJF months), MDP climatology values

### Comparison 2, dry season (DJF months), WSD climatology